# Case count dependent measures
This notebook contains the experiment to investigate the **effect of measures that become active only if the number of new cases per week exceeds a certain threshold** as currently discussed and implemented by the German government (source: https://www.zdf.de/nachrichten/politik/landkreise-lockdown-zahlen-karte-100.html). Whenever the threshold was exceeded in the previous week, site closures and/or social distancing measures become active for one week. 
Alternatively one can specify intervention times (e.g. sundays in Germany) at which the policy will be revised. The isolation of positively tested people remains active throughout the simulation.


The notebook is organized as follows:

* In Section 1, we define all the simulation parameters.
* In Section 2, we run all the simulations needed for the experiment.
* In Section 3, we plot the results.

To just regenerate the figures from a summary file containing all the simulation objects of the experiment, you can skip Section 2, and only run the cells in Sections 1 & 3. 

---

#### Import libs

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [ ]:
import numpy as np
import pickle, math
import pandas as pd
import multiprocessing

In [ ]:
from lib.measures import *
from lib.experiment import run_experiment, save_summary, load_summary
from lib.calibration_settings import settings_lockdown_dates

## 1. General settings  

#### Set the random seed for reproducibility

In [ ]:
# Choose random seed
c = 0
np.random.seed(c)
TO_HOURS = 24.0

# Define prefix string used to save plots
expstr = 'conditional-measures'

#### Set the number of roll-outs to simulate

In [ ]:
random_repeats = 1 # Set to at least 40 to obtain stable results

#### Set the time to simulate

In [ ]:
end_date = '2020-07-31'

#### Define locations, mobility settings used for simulation, and experiment parameters

In [ ]:
locs = {
    'GER': {
        'TU': 'lib/mobility/Tubingen_settings_10.pk',
        'KL': 'lib/mobility/Kaiserslautern_settings_10.pk',
        'RH': 'lib/mobility/Ruedesheim_settings_10.pk', 
        'TR': 'lib/mobility/Tirschenreuth_settings_10.pk', 
    },
    'CH': {
        'VD': 'lib/mobility/Lausanne_settings_10.pk', 
        'LU': 'lib/mobility/Lucerne_settings_5.pk', 
        'TI': 'lib/mobility/Locarno_settings_2.pk', 
        'JU': 'lib/mobility/Jura_settings_10.pk',
    }
}

seed_summary_path = None

In [ ]:
# experiment parameters

# Site closures and social distancing measures become active if the number of positive tests exceeds threshold
threshold_tests_per_100k = 50
is_measure_active_initially = True

# Times of possible interventions, if None, measures can become active at any time
intervention_times = None
#start_date = settings_lockdown_dates['GER']['end']
#max_time = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days * TO_HOURS
#intervention_times = list(np.arange(0, max_time, 7.0 * TO_HOURS))

# Experiment a): Closure of educational and social sites only
# Experiment b): Closure of educational and social sites + social distancing
experiments = ['exp a', 'exp b']

In [ ]:
# Check consistency of parameter choice
from lib.inference import get_test_capacity, get_scaled_test_threshold
from lib.mobilitysim import MobilitySimulator

for country, areas in locs.items():
    for area, mob_settings in areas.items():

        
        with open(mob_settings, 'rb') as fp:
            obj = pickle.load(fp)
        mob = MobilitySimulator(**obj)

        scaled_test_capacity = get_test_capacity(country, area, mob)
        scaled_test_threshold = get_scaled_test_threshold(threshold_tests_per_100k, mob)
        print(f'Country: {country}, area: {area}')
        print(f'Threshold of new cases per week: {scaled_test_threshold}')
        print(f'Test capacity per week: {7*scaled_test_capacity}')

## 2. Run the simulations

Use settings as above and simulate in the future with additional measures

**WARNING: the following cells might take a long time to run depending of the parameters defined above!**

In [ ]:
for country, areas in locs.items():
    for area, mob_settings in areas.items():
        s = []
        
        # start simulation when lockdown ends
        start_date = settings_lockdown_dates[country]['end']
        
        # load social distancing parameter
        p_stay_home = 0.7 # TBD

        # baseline
        baseline = run_experiment(
            country=country, 
            area=area, 
            mob_settings=mob_settings,
            start_date=start_date, 
            end_date=end_date, 
            measure_list=[], 
            random_repeats=random_repeats,
            test_update=None, 
            seed_summary_path=seed_summary_path)
        s.append(baseline)

        print(f'{country} {area} baseline done.', flush=True)

        # experiment
        with open(mob_settings, 'rb') as fp:
            obj = pickle.load(fp)
        mob = MobilitySimulator(**obj)
        scaled_test_threshold = get_scaled_test_threshold(threshold_tests_per_100k, mob)


        for experiment in experiments:
            
            # additional measures
            max_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days
            m = [
                # Site closures conditional on weekly new cases
                UpperBoundCasesBetaMultiplier(t_window=Interval(0, max_days * TO_HOURS), 
                                              beta_multiplier={'education': 0.0, 
                                                               'social': 0.0,
                                                               'bus_stop': 1.0, 
                                                               'office': 1.0,
                                                               'supermarket': 1.0},
                                              max_pos_tests_per_week=scaled_test_threshold, 
                                              intervention_times=intervention_times,
                                              init_active=is_measure_active_initially)
                ]
            
            if experiment == 'exp b':
                m.append(
                        # Social distancing conditional on weekly new cases
                        UpperBoundCasesSocialDistancing(t_window=Interval(0, max_days * TO_HOURS), 
                                                        p_stay_home=p_stay_home, 
                                                        max_pos_tests_per_week=scaled_test_threshold, 
                                                        intervention_times=intervention_times,
                                                        init_active=is_measure_active_initially)
                        )
                
            # update testing params
            test_update = None
            
            # run
            res = run_experiment(
                country=country, 
                area=area, 
                mob_settings=mob_settings,
                start_date=start_date, 
                end_date=end_date, 
                measure_list=m, 
                random_repeats=random_repeats,
                test_update=None, 
                seed_summary_path=seed_summary_path)
            
            s.append(res)
            
            print(f'{country} {area} experiment {experiment}) done.', flush=True)
        
        # save results
        save_summary(s, f'state-{expstr}--{country}-{area}.pk')

#### Find the dates on which measures became active/inactive

In [ ]:
from itertools import chain
def get_lockdown_dates(summary):
    hist = list(summary.measure_list[0].find(UpperBoundCasesBetaMultiplier, t=1).intervention_history)
    
    lockdowns = [hist[0][:2]]
    j = 0
    for k in range(len(hist)):
        if k > j:
            # If the time between two lock down periods is less than a day we count it as one lockdown
            if hist[k][0] - lockdowns[j][1] < 24.0:
                lockdowns[j] = (lockdowns[j][0], hist[k][1])
            else:
                lockdowns.append(hist[k][0:2])
                j += 1
                
    lockdown_labels = []
    for _ in lockdowns:
        lockdown_labels.append(['active', 'inactive'])

    lockdowns = np.asarray(lockdowns) / 24.0
    lockdowns = list(chain.from_iterable(lockdowns))
    lockdown_labels = list(chain.from_iterable(lockdown_labels))
    return lockdowns, lockdown_labels

In [ ]:
def get_lockdown_info(summary):
    lockdowns = {key: [] for key in experiments}
    lockdown_labels = {key: [] for key in experiments}
    whichsim = 1
    for k in lockdowns.keys():
        dates, labels = get_lockdown_dates(summary[whichsim])
        lockdowns[k] = dates
        lockdown_labels[k] = labels
        whichsim +=1
    return lockdowns, lockdown_labels

## 3. Plot the results

Import libs

In [ ]:
from lib.plot import Plotter

Create figures from summaries

In [ ]:
for country, areas in locs.items():
    for area in areas:
        s = load_summary(f'state-{expstr}--{country}-{area}.pk')
        lockdowns, lockdown_labels = get_lockdown_info(s)
        plotter = Plotter()
        titles = ['no case dep. measures'] + ['site closures'] + ['site closures + social distancing']
        fn = f'{expstr}--{country}-{area}'
        print(fn)
        plotter.compare_total_infections(
            s, 
            titles=titles, 
            figtitle=r'Case number dependent measures',
            filename=fn, 
            lockdown_at=lockdowns,
            lockdown_label=lockdown_labels,
            lockdown_label_y={'exp a': 300, 'exp b': 150},
            start_date=settings_lockdown_dates[country]['end'],
            figsize=(6,4), acc=500, 
            ymax=500, errorevery=14)